# Proyecto Final: Procesamiento de Lenguaje Natural

## Definición del proyecto

El objetivo de este proyecto es utilizar una Red Neuronal Recurrente usando LSTM (Long-Short Term Memory) para hacer traducción de Inglés a Italiano. En los datos de entrenamiento que se nos proporcionan tenemos un par de frases; la frase en inglés y su correspondiente traducción a italiano. 

Para lograr esto, utilizaremos una red "Sequence to Sequence" en el que tendremos dos redes neuronales recurrentes para transformar una secuencia en otra; la primera nos servira como un "Encoder" para transofrmar una entrada de text a un vector mientras que la segunda nos servira para poder

![Red Sequence-Sequence](./imagenes/encoder-decoder.png)

## Requerimientos

Para poder realizar este proyecto decidimos usar PyTorch para poder utilizar tensores y aprovechar el GPU de la computadora para el procesamiento (algo que con numpy no se puede hacer). La versión de Pytorch que utilizamos para este proyecto tiene las siguientes caracteristicas:
 - Version 1.1
 - Windows 10
 - Cuda 10.1
 - Python 3.7
 - Conda

In [ ]:
#conda install pytorch torchvision cudatoolkit=10.0 -c pytorch

## Imports

Importamos todas la bibliotecas que vamos a estar utilizando. En este proeycto utilizaremos algunas librerias generales como lo son `string` y `re` que son de uso común durante cualquier proyecto de procesamiento de texto.

In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

# Cargamos torch
import torch
# Cargamos la biblioteca de redes neuronales de pytorch
import torch.nn as nn
# Biblioteca de optimización de algoritmos
from torch import optim
# Funciones para entrenar la red neuronal.
import torch.nn.functional as F

# Buscamos si el dispositivo donde se ejecuita el algoritmo tiene cuda instalado
# en caso de que no lo tenga usará el CPU para el entrenamiento.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Carga de texto
En este proyecto el proceso de carga de texto se realiza utilizando pares de frases la primera en inglés seguido de un tab y despues en italiano.

```
we therefore respect whatever parliament may decide 	quindi noi rispettiamo le eventuali decisioni in materia del parlamento

```

Como vimos en clase la primera etapa será construir nuestros vectores one hot de las palabras que contiene el corpus de entrenamiento. Para esto generaremos un indice unico para cada una de las palabras el cual utilizará para ubicarse dentro del vector. 

![One Hot](./imagenes/onehot.png)

Desarrollamos una clase llamada Lenguaje, el cual contendrá el las funciones necesarias para pasar de la palabra al indíce y del indice de regreso a la palabra utilizando diccionarios. 

## Referencias
 
 - Basile, Pierpaolo, et al. “Bi-Directional LSTM-CNNs-CRF for Italian Sequence Labeling.” Proceedings of the Fourth Italian Conference on Computational Linguistics CLiC-It 2017, 2017, pp. 18–23., doi:10.4000/books.aaccademia.2339.
 - Raval, Siraj. “Recurrent Neural Network - The Math of Intelligence (Week 5).” YouTube, YouTube, 19 July 2017, www.youtube.com/watch?v=BwmddtPFWtA.
 - Raval, Siraj. “LSTM Networks - The Math of Intelligence (Week 8).” YouTube, YouTube, 9 Aug. 2017, www.youtube.com/watch?v=9zhrxE5PQgY. 
 - Trask, Andrew. “Anyone Can Learn To Code an LSTM-RNN in Python (Part 1: RNN).” Anyone Can Learn To Code an LSTM-RNN in Python (Part 1: RNN) - i Am Trask, iamtrask.github.io/2015/11/15/anyone-can-code-lstm/.
 - PyTorch. “Translation with a Sequence to Sequence Network and Attention.” Translation with a Sequence to Sequence Network and Attention - PyTorch Tutorials 1.1.0 Documentation, pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html.


